<a href="https://colab.research.google.com/github/tarakantaacharya/TTS_ODIA/blob/main/Preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

In [3]:
import json
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
import librosa

# Function to preprocess the audio (example: resample and normalize)
def preprocess_audio(audio_data, sampling_rate, target_sr=16000):
    # Resample the audio to a target sample rate (if needed)
    if sampling_rate != target_sr:
        audio_data = librosa.resample(audio_data, orig_sr=sampling_rate, target_sr=target_sr)  # Corrected
        sampling_rate = target_sr  # Update the sampling rate to target

    # Normalize the audio (scaling the waveform between -1 and 1)
    audio_data = librosa.util.normalize(audio_data)

    return audio_data, sampling_rate

# Function to preprocess the text (tokenize)
def preprocess_text(text, tokenizer):
    # Tokenize the text into input IDs
    tokens = tokenizer(text, return_tensors="pt")
    return tokens['input_ids'].squeeze(0).tolist()  # Get tokenized text as list of IDs

# Function to preprocess data from the dataset
def preprocess_data_from_dataset(dataset, index, tokenizer):
    # Extract the audio data and sampling rate
    audio_data = dataset[index]['audio']['array']
    sampling_rate = dataset[index]['audio']['sampling_rate']

    # Preprocess the audio
    norm_audio_data, processed_sr = preprocess_audio(audio_data, sampling_rate)

    # Preprocess the text
    text = dataset[index]['text']
    tokenized_text = preprocess_text(text, tokenizer)

    # Return the preprocessed audio and tokenized text
    return norm_audio_data.tolist(), tokenized_text

# Load the dataset
dataset = load_dataset("SPRINGLab/IndicTTS_Odia")

# Load the tokenizer for the language model
tokenizer = tokenizer

processed_data = []

batch_size = 50  # Load 50 rows at a time

# Iterate over dataset in batches of 50
for i in range(0, len(dataset['train'])//15, batch_size):
    batch = dataset['train'].select(range(i, min(i + batch_size, len(dataset['train']))))  # Select batch

    for example in batch:
        norm_audio_data, tokenized_text = preprocess_data_from_dataset(batch, 0, tokenizer)

        processed_data.append({
            "audio": norm_audio_data,
            "text": tokenized_text
        })

    print(f"Processed {min(i + batch_size, len(dataset['train']))} rows out of {len(dataset['train'])}")

# Store the processed data as a JSON file
with open("processed_data.json", "w") as json_file:
    json.dump(processed_data, json_file)

print("Data has been saved to processed_data.json.")

README.md:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11564 [00:00<?, ? examples/s]

Processed 50 rows out of 11564
Processed 100 rows out of 11564
Processed 150 rows out of 11564
Processed 200 rows out of 11564
Processed 250 rows out of 11564
Processed 300 rows out of 11564
Processed 350 rows out of 11564
Processed 400 rows out of 11564
Processed 450 rows out of 11564
Processed 500 rows out of 11564
Processed 550 rows out of 11564
Processed 600 rows out of 11564
Processed 650 rows out of 11564
Processed 700 rows out of 11564
Processed 750 rows out of 11564
Processed 800 rows out of 11564
Data has been saved to processed_data.json.


In [4]:
import pandas as pd
df = pd.read_json('processed_data.json')
len(df)

800

In [5]:
import numpy as np
np.array(df.iloc[0]['audio']).shape

(47226,)

In [6]:
import numpy as np

# Function to pad audio to the maximum length
def pad_audio(audio, target_length):
    # If the audio is shorter than the target length, pad it with zeros
    if len(audio) < target_length:
        padding = np.zeros(target_length - len(audio))
        return np.concatenate([audio, padding])
    else:
        # If the audio is longer, truncate it
        return audio[:target_length]

# Find the maximum audio length
max_length = max([len(audio) for audio in df['audio']])

# Pad the audio to the maximum length
df['padded_audio'] = df['audio'].apply(lambda x: pad_audio(x, max_length))

# Check the shape of padded audio
print(df['padded_audio'].apply(lambda x: len(x)).value_counts())

padded_audio
112709    800
Name: count, dtype: int64


In [7]:
print(type(df['audio']))

<class 'pandas.core.series.Series'>


In [8]:
print(type(df['text']))

<class 'pandas.core.series.Series'>


In [9]:
import numpy as np

# Function to pad audio to the maximum length
def pad_text(text, target_length):
    # If the audio is shorter than the target length, pad it with zeros
    if len(text) < target_length:
        padding = np.zeros(target_length - len(text))
        return np.concatenate([text, padding])
    else:
        # If the audio is longer, truncate it
        return text[:target_length]

# Find the maximum audio length
max_length = max([len(audio) for audio in df['text']])

# Pad the audio to the maximum length
df['padded_text'] = df['text'].apply(lambda x: pad_text(x, max_length))

# Check the shape of padded audio
print(df['padded_text'].apply(lambda x: len(x)).value_counts())

padded_text
66    800
Name: count, dtype: int64


In [10]:
df.columns

Index(['audio', 'text', 'padded_audio', 'padded_text'], dtype='object')

In [11]:
print(df.iloc[0])

audio           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
text            [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...
padded_audio    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
padded_text     [2.0, 41.0, 62688.0, 1.0, 41.0, 1.0, 62827.0, ...
Name: 0, dtype: object


In [12]:
print(df.head())

                                               audio  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                text  \
0  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...   
1  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...   
2  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...   
3  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...   
4  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...   

                                        padded_audio  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [13]:
new_preprocess_data = df.drop(columns=['text','audio'])

In [14]:
new_preprocess_data.head()

,padded_audio,padded_text
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 41.0, 62688.0, 1.0, 41.0, 1.0, 62827.0, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 41.0, 62688.0, 1.0, 41.0, 1.0, 62827.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 41.0, 62688.0, 1.0, 41.0, 1.0, 62827.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 41.0, 62688.0, 1.0, 41.0, 1.0, 62827.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 41.0, 62688.0, 1.0, 41.0, 1.0, 62827.0, ..."


In [15]:
type(new_preprocess_data['padded_text'].iloc[1])

numpy.ndarray

In [16]:
new_preprocess_data.to_json("new_preprocess_data.json")

In [17]:
new_preprocess_data.columns

Index(['padded_audio', 'padded_text'], dtype='object')

In [18]:
print(new_preprocess_data.iloc[0])

padded_audio    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
padded_text     [2.0, 41.0, 62688.0, 1.0, 41.0, 1.0, 62827.0, ...
Name: 0, dtype: object


In [19]:
import numpy as np

In [20]:
type(new_preprocess_data['padded_audio'].iloc[1])

numpy.ndarray

In [21]:
type(new_preprocess_data['padded_text'].iloc[1])

numpy.ndarray

In [22]:
# Function to check the shape of audio arrays
def check_audio_shape(df):
    # Create a dictionary to store the counts of each shape
    shape_counts = {}

    for audio in df['padded_audio']:
        # Get the shape of the audio (assuming each audio is a numpy array or list)
        shape = len(audio)

        # Update the count for the shape
        if shape in shape_counts:
            shape_counts[shape] += 1
        else:
            shape_counts[shape] = 1

    return shape_counts

# Get the shape counts
shape_counts = check_audio_shape(new_preprocess_data)

# Print the result
for shape, count in shape_counts.items():
    print(f"Audio with shape {shape}: {count} files")

Audio with shape 112709: 800 files


In [23]:
# Function to check the shape of audio arrays
def check_text_shape(df):
    # Create a dictionary to store the counts of each shape
    shape_counts = {}

    for text in df['padded_text']:
        # Get the shape of the audio (assuming each audio is a numpy array or list)
        shape = len(text)

        # Update the count for the shape
        if shape in shape_counts:
            shape_counts[shape] += 1
        else:
            shape_counts[shape] = 1

    return shape_counts

# Get the shape counts
shape_counts = check_text_shape(new_preprocess_data)

# Print the result
for shape, count in shape_counts.items():
    print(f"Text with shape {shape}: {count} files")

Text with shape 66: 800 files


---